In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import serial

%matplotlib qt

ser = serial.Serial("COM3", 921600)

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot([], [])
line2, = ax.plot([], [])
line3, = ax.plot([], [])
line4, = ax.plot([], [])
line1.set_xdata(np.linspace(0, 307, 307))
line2.set_xdata(np.linspace(0, 307, 307))
line3.set_xdata(np.linspace(0, 307, 307))
line4.set_xdata(np.linspace(0, 307, 307))

while True:
    echo = []   
    for i in range(4):
        idata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]
        qdata = [int.from_bytes(ser.read(3), byteorder='big', signed="True") for x in range(256)]

        micdata = list(map(lambda x: x[0]+(x[1]*1j), list(zip(idata, qdata))))

        pattern = np.array([[1] * 4 if(int(i)==1) else [-1] * 4 for i in bin(0x1f35)[2:]]).flatten()
        conv = np.convolve(micdata, pattern)

        amp = [np.sqrt(x.imag**2 + x.real**2) for x in conv]
        phase = [np.arctan(x.real/x.imag) for x in conv]

        echo.append([amp, phase])
    
    line1.set_ydata(echo[0][0])
    line2.set_ydata(echo[1][0])
    line3.set_ydata(echo[2][0])
    line4.set_ydata(echo[3][0])
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()